<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_de_Tensorflow-2.0_Guia_completa_para_el_Nuevo_Tensorflow/blob/master/Collab_13_Preparar_un_modelo_de_TensorFlow_para_m%C3%B3viles_con_TensorFlow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalar las dependencias y configurar el entorno de trabajo

In [ ]:
!pip install tensorflow-gpu==1.13.1

## Paso 2: Importar las dependencias

In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'2.3.0'

## Paso 3: Pre procesado de datos

### Cargar el dataset de FashionMNIST

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Normalizar las imágenes

In [3]:
X_train = X_train / 255.
X_test = X_test / 255.

### Redimensionar las imágenes

In [4]:
X_train.shape

(60000, 28, 28)

In [5]:
#convertimos las imagenes un un vector columna una solo dimencion
X_train = X_train.reshape(-1, 28*28) # -1  es para solo tener una profundida
X_test = X_test.reshape(-1, 28*28)

In [ ]:
X_train.shape  # solo es ya un vector muy largo

## Paso 4: Construir el modelo

### Defining the model

In [7]:
model = tf.keras.models.Sequential()

### Definir el modelo

In [8]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))# capa de olvido para no se tenga overfitin
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

### Compilar el modelo

In [9]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Entrenar el modelo

In [10]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5287 - sparse_categorical_accuracy: 0.8123
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3967 - sparse_categorical_accuracy: 0.8560
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3635 - sparse_categorical_accuracy: 0.8667
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3445 - sparse_categorical_accuracy: 0.8738
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3278 - sparse_categorical_accuracy: 0.8794


### Evaluar el modelo

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Precisión del conjunto de test: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 73us/sample - loss: 0.3446 - sparse_categorical_accuracy: 0.8778
Precisión del conjunto de test: 0.8777999877929688


## Paso 5: Convertir el modelo a la versión de TensorFlow Lite

### Guardar el modelo

In [ ]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Crear el TFLite Converter

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


### Convertir el modelo

In [ ]:
tflite_model = converter.convert()

### Guardar la versión de TFLite version del modelo

In [ ]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)